In [1]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete
# Run-time: Unknown

# Dependencies:
#  - utils.py for catch

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=10) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=10) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=10) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=10) # kavi IBG paper-trade

In [9]:
import numpy as np
import pandas as pd
from itertools import product
import datetime

import utils # for catch function

# sd multiple for band
sigma = 2       # 2 sigma is about 95% probability
penalty = 1.8   # e.g. 1.2 is 20% above

# market
exchange = 'NSE'

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

equities = list(pd.read_pickle('./zdata/df_nse_eq_symbols.pkl').ibSymbol)  # taken from pickle. To be changed to take from df_paisa - differentiating stocks and equities

def df_e(symbol):
    '''builds df for equities, based on 1Scrip_Program
    Arg: (symbol) as string
    Returns: dataframe of options '''
    
    contract = Stock(symbol, 'NSE')
    ib.qualifyContracts(contract)

    #... Get volatility, hi52 and lo52
    duration = '12 M'
    size = '1 day'
    bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                         durationStr=duration, barSizeSetting=size, 
                         whatToShow='MIDPOINT', useRTH=True, 
                         formatDate=1, keepUpToDate=True)
    stDev = np.std(a=[b.close for b in bars], ddof=0)

    hi52 = max([b.high for b in bars])
    lo52 = min([b.low for b in bars])

    meanPrice = np.mean([b.close for b in bars])

    #... Get the lot and margin
    lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
    margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

    #... Get the option chain tickers
    chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                          futFopExchange='', 
                          underlyingConId=contract.conId, underlyingSecType=contract.secType)

    expiries = set(*[c.expirations for c in chains])

    cds = [ib.reqContractDetails(Option(symbol, e, exchange='NSE')) for e in expiries]

    options = [c.contract for cs in cds for c in cs]
    
    ib.sleep(0.1)

    tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]

    # keep only those tickers with underlying prices
    lib_t = {t: utils.catch(lambda: t.modelGreeks.undPrice) for t in tickers}
    und_t = [k for k, v in lib_t.items() if v is not None]


    #... Weed out unwanted SDs and make the option chain dataframe
    # keep only the Ps and Cs outside the sigma band (95% probability)
    tgt_ts = [t for t in und_t 
    if ((t.contract.strike < t.modelGreeks.undPrice - stDev*sigma) & (t.contract.right == 'P')) | 
    ((t.contract.strike > t.modelGreeks.undPrice + stDev*sigma) & (t.contract.right == 'C'))]

    # ... Build the dataframe
    ts = [(t.contract.conId, t.contract.symbol, t.contract.lastTradeDateOrContractMonth, t.contract.strike, t.contract.right, t.modelGreeks.undPrice,
      t.contract.localSymbol, t.bid, t.bidSize, t.ask, t.askSize, t.close, t.modelGreeks.impliedVol, t.modelGreeks.delta, t.modelGreeks.optPrice, 
      t.modelGreeks.pvDividend, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta, lot, margin, hi52, lo52, meanPrice, stDev, t) for t in tgt_ts]

    cols = ['conId', 'ibSymbol', 'expiry', 'strike', 'right', 'undPrice', 
            'localSymbol', 'bid', 'bidSize', 'ask', 'askSize', 'close', 'impliedVol', 'delta', 'optPrice', 
            'pvDividend', 'gamma', 'vega', 'theta', 'lot', 'margin', 'hi52', 'lo52', 'meanPrice', 'stDev', 'ticker']
    df = pd.DataFrame(ts, columns=cols).sort_values(by=['expiry', 'strike'], ascending=False)

    # Expected price is rounded up to 0.05 centrs after putting the penalty
    df['expPrice'] = round(df[['bid', 'ask', 'close']].max(axis=1)*penalty * 2, 1)/2

    df['dte'] = (pd.to_datetime(df.expiry) - datetime.datetime.now()).dt.days

    df['rom'] = (df.expPrice*df.lot)/df.margin*252/df.dte
    
    return df

In [12]:
%%time
dfs = [df_e(s) for s in equities[0:10]]

Wall time: 12min 5s


In [16]:
df = pd.concat(dfs, axis=0).reset_index(drop=True)

In [20]:
df.sort_values(by=['rom'], ascending=False)

,conId,ibSymbol,expiry,strike,right,undPrice,localSymbol,bid,bidSize,ask,...,lot,margin,hi52,lo52,meanPrice,stDev,ticker,expPrice,dte,rom
217,335439894,ADANIPOWE,20181227,26.0,P,50.10,ADANIPOWER18DEC26PE,-1.00,0,2.50,...,20000,359675.00,58.325,15.175,31.436458,10.608533,"Ticker(contract=Option(conId=335439894, symbol...",4.50,2,31.528463
466,335443004,AMBUJACEM,20181227,275.0,C,217.65,AMBUJACEM18DEC275CE,-1.00,0,3.00,...,2500,96706.25,279.975,188.400,227.201875,21.876768,"Ticker(contract=Option(conId=335443004, symbol...",5.40,2,17.589349
304,335397877,ALBK,20181227,26.0,P,45.75,ALBK18DEC26PE,-1.00,0,0.60,...,11000,90007.50,74.275,33.175,47.086667,9.279703,"Ticker(contract=Option(conId=335397877, symbol...",1.10,2,16.938588
305,335397870,ALBK,20181227,25.0,P,45.75,ALBK18DEC25PE,-1.00,0,0.60,...,11000,90007.50,74.275,33.175,47.086667,9.279703,"Ticker(contract=Option(conId=335397870, symbol...",1.10,2,16.938588
303,335397887,ALBK,20181227,27.0,P,45.75,ALBK18DEC27PE,-1.00,0,0.60,...,11000,90007.50,74.275,33.175,47.086667,9.279703,"Ticker(contract=Option(conId=335397887, symbol...",1.10,2,16.938588
652,335375151,APOLLOTYR,20181227,165.0,P,226.25,APOLLOTYRE18DEC165PE,-1.00,0,3.00,...,3000,122760.00,307.175,192.100,257.655000,25.412401,"Ticker(contract=Option(conId=335375151, symbol...",5.40,2,16.627566
653,335375141,APOLLOTYR,20181227,160.0,P,226.25,APOLLOTYRE18DEC160PE,-1.00,0,3.00,...,3000,122760.00,307.175,192.100,257.655000,25.412401,"Ticker(contract=Option(conId=335375141, symbol...",5.40,2,16.627566
654,335375132,APOLLOTYR,20181227,155.0,P,226.25,APOLLOTYRE18DEC155PE,-1.00,0,3.00,...,3000,122760.00,307.175,192.100,257.655000,25.412401,"Ticker(contract=Option(conId=335375132, symbol...",5.40,2,16.627566
650,335375166,APOLLOTYR,20181227,175.0,P,226.25,APOLLOTYRE18DEC175PE,-1.00,0,3.00,...,3000,122760.00,307.175,192.100,257.655000,25.412401,"Ticker(contract=Option(conId=335375166, symbol...",5.40,2,16.627566
655,335375126,APOLLOTYR,20181227,150.0,P,226.25,APOLLOTYRE18DEC150PE,-1.00,0,3.00,...,3000,122760.00,307.175,192.100,257.655000,25.412401,"Ticker(contract=Option(conId=335375126, symbol...",5.40,2,16.627566
